### 1. 협업 필터링 (Collaborative Filtering)

1) 협업 필터링이란?
- 사용자 행동 패턴을 기반으로 새로운 아이템을 추천하는 방식
- 주어진 데이터가 사용자-아이템 상호작용 데이터(User-Item Interaction Matrix)인 경우

2) 협업필터링의 종류
- 사용자 기반(User-Based) 협업 필터링
- 아이템 기반(Item-Based) 협업 필터링

#### 1-1. 사용자 기반 협업 필터링 구현

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 사용자-아이템 평점 데이터 생성
ratings = pd.DataFrame([
    [5, 3, 0, 1],
    [4, 0, 4, 2],
    [1, 1, 0, 5],
    [0, 0, 4, 4]
], index=["User 1", "User 2", "User 3", "User 4"],
   columns=["Item A", "Item B", "Item C", "Item D"])

print("🔹 사용자-아이템 행렬:\n", ratings)
print("\n")

# 사용자 간 유사도 계산 (코사인 유사도)
user_similarity = cosine_similarity(ratings.fillna(0))
user_sim_df = pd.DataFrame(user_similarity, index=ratings.index, columns=ratings.index)

print("🔹 사용자 간 유사도:\n", user_sim_df)
print("\n")

# User 1과 가장 유사한 사용자 찾기
similar_users = user_sim_df["User 1"].sort_values(ascending=False)[1:]
print("🔹 User 1과 가장 유사한 사용자:", similar_users.idxmax())
print("\n")

# 가장 유사한 사용자의 아이템 추천
most_similar_user = similar_users.idxmax()
recommended_items = ratings.loc[most_similar_user][ratings.loc["User 1"] == 0].sort_values(ascending=False)
print("🔹 User 1에게 추천할 아이템:\n", recommended_items)

🔹 사용자-아이템 행렬:
         Item A  Item B  Item C  Item D
User 1       5       3       0       1
User 2       4       0       4       2
User 3       1       1       0       5
User 4       0       0       4       4


🔹 사용자 간 유사도:
           User 1    User 2    User 3    User 4
User 1  1.000000  0.619780  0.422890  0.119523
User 2  0.619780  1.000000  0.449050  0.707107
User 3  0.422890  0.449050  1.000000  0.680414
User 4  0.119523  0.707107  0.680414  1.000000


🔹 User 1과 가장 유사한 사용자: User 2


🔹 User 1에게 추천할 아이템:
 Item C    4
Name: User 2, dtype: int64


#### 1-2. 아이템 기반 협업 필터링 구현

In [2]:
# 아이템 간 유사도 계산
item_similarity = cosine_similarity(ratings.T.fillna(0))
item_sim_df = pd.DataFrame(item_similarity, index=ratings.columns, columns=ratings.columns)

print("🔹 아이템 간 유사도:\n", item_sim_df)
print("\n")

# 특정 아이템과 유사한 아이템 찾기
target_item = "Item C"
similar_items = item_sim_df[target_item].sort_values(ascending=False)[1:]

print(f"🔹 '{target_item}'와 유사한 아이템:\n", similar_items.idxmax())

🔹 아이템 간 유사도:
           Item A    Item B    Item C    Item D
Item A  1.000000  0.780720  0.436436  0.409514
Item B  0.780720  1.000000  0.000000  0.373002
Item C  0.436436  0.000000  1.000000  0.625543
Item D  0.409514  0.373002  0.625543  1.000000


🔹 'Item C'와 유사한 아이템:
 Item D


### 2. 행렬 분해(Matrix Factorization) 기반 추천 시스템

#### 1) 행렬 분해(Matrix Factorization, MF)란?
- 사용자-아이템 행렬을 저차원 벡터로 분해하여 잠재 요인(Latent Factors)을 학습하는 방법.

✅ 주요 행렬 분해 알고리즘

- SVD (Singular Value Decomposition) : 가장 기본적인 행렬 분해 방식
- NMF (Non-negative Matrix Factorization) : 음수가 없는 데이터에서 유용
- ALS (Alternating Least Squares) : 대규모 데이터에서 강력한 성능
- Deep Learning 기반 행렬 분해 : 신경망을 활용한 확장된 방법

#### 2-1. SVD

##### SVD 추천 방식

In [4]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

# 사용자-아이템 행렬 (예제 데이터)
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

# SVD 행렬 분해 (2개의 잠재 요인 사용)
svd = TruncatedSVD(n_components=2)
U = svd.fit_transform(R)  # 사용자 잠재 요인 행렬
Sigma = np.diag(svd.singular_values_)  # 특이값 행렬
V = svd.components_  # 아이템 잠재 요인 행렬

In [5]:
U

array([[ 3.94592157,  4.16887476],
       [ 2.68400116,  2.76010383],
       [ 4.65943967, -0.84914207],
       [ 3.61265495, -0.69003845],
       [ 4.90266747, -3.55087873]])

In [6]:
Sigma

array([[9.03171974, 0.        ],
       [0.        , 6.22925557]])

In [7]:
V

array([[ 0.47488998,  0.26234348,  0.3005118 ,  0.78444124],
       [ 0.78203025,  0.20891356, -0.45754472, -0.36801718]])

In [8]:
# 예측 평점 행렬 복원
R_pred = np.dot(np.dot(U, Sigma), V)
print("=== SVD 추천 예측 행렬 ===")
print(np.round(R_pred, 2))

=== SVD 추천 예측 행렬 ===
[[37.23 14.77 -1.17 18.4 ]
 [24.96  9.95 -0.58 12.69]
 [15.85  9.94 15.07 34.96]
 [12.13  7.66 11.77 27.18]
 [ 3.73  7.   23.43 42.87]]


##### 라이브러리 활용

In [3]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split

# 데이터 준비
ratings_dict = {
    "userID": [1, 1, 1, 2, 2, 3, 3, 4, 4, 4],
    "itemID": [1, 2, 3, 1, 3, 2, 3, 1, 2, 3],
    "rating": [5, 3, 4, 4, 5, 2, 3, 5, 4, 4]
}
df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

# 데이터 분할 (훈련/테스트)
trainset, testset = train_test_split(data, test_size=0.2)

# SVD 모델 학습
model = SVD(n_factors=10, n_epochs=20, lr_all=0.005, reg_all=0.02)
model.fit(trainset)

# 예측 수행
predictions = model.test(testset)

# 평가
cross_validate(model, data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1893  0.9003  1.2347  1.3547  0.6978  0.8753  0.4152  
MAE (testset)     0.1619  0.8871  1.2272  1.0344  0.6123  0.7846  0.3705  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.18928193, 0.90025861, 1.23467293, 1.35472279, 0.69778086]),
 'test_mae': array([0.16187875, 0.88710515, 1.22716039, 1.03439743, 0.61230039]),
 'fit_time': (0.0, 0.0, 0.0, 0.0, 0.0),
 'test_time': (0.0, 0.0, 0.0, 0.0, 0.0)}

- SVD를 사용한 특정 사용자에게 추천

In [4]:
# 특정 사용자에게 추천 수행
user_id = 1
item_ids = df['itemID'].unique()
preds = [model.predict(user_id, iid) for iid in item_ids]

# 예측 평점이 높은 순으로 정렬
sorted_preds = sorted(preds, key=lambda x:x.est, reverse=True)

# 추천 아이템 출력
print("🔹 사용자 1에게 추천할 아이템:")
for pred in sorted_preds:
    print(f"아이템 {pred.iid} | 예상 평점: {pred.est:.2f}")

🔹 사용자 1에게 추천할 아이템:
아이템 1 | 예상 평점: 4.25
아이템 3 | 예상 평점: 4.06
아이템 2 | 예상 평점: 3.74


#### 2-2. ALS (Alternating Least Squares)

ALS는 교대 최소제곱법(Alternating Least Squares)을 이용하여 사용자 행렬과 아이템 행렬을 최적화 하는 방식입니다.

##### ALS 추천 방식

In [9]:
def als_recommendation(R, num_features=2, iterations=10, alpha=0.01):
    num_users, num_items = R.shape
    U = np.random.rand(num_users, num_features)  # 사용자 행렬 초기화
    V = np.random.rand(num_items, num_features)  # 아이템 행렬 초기화

    for _ in range(iterations):
        # U 업데이트: V를 고정하고 최적화
        for i in range(num_users):
            U[i] = np.linalg.solve(np.dot(V.T, V) + alpha * np.eye(num_features), np.dot(V.T, R[i, :].T))

        # V 업데이트: U를 고정하고 최적화
        for j in range(num_items):
            V[j] = np.linalg.solve(np.dot(U.T, U) + alpha * np.eye(num_features), np.dot(U.T, R[:, j]))

    # 예측 행렬 복원
    R_pred = np.dot(U, V.T)
    return R_pred

# ALS 추천 결과 계산
R_als_pred = als_recommendation(R)

print("=== ALS 추천 예측 행렬 ===")
print(np.round(R_als_pred, 2))

=== ALS 추천 예측 행렬 ===
[[ 5.13  1.9  -0.72  1.56]
 [ 3.43  1.28 -0.46  1.09]
 [ 1.55  1.05  1.79  3.96]
 [ 1.18  0.8   1.4   3.09]
 [-0.45  0.54  3.1   5.15]]


##### 라이브러리 활용한 암묵적 피드백 기반 추천 (Implicit 라이브러리)

In [6]:
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

# 🎯 사용자-아이템 상호작용 데이터
user_item_matrix = np.array([
    [5, 0, 3, 0, 2],  
    [4, 0, 0, 1, 3],  
    [1, 1, 0, 5, 0],  
    [0, 0, 4, 4, 0]   
])

# 🎯 희소 행렬 변환
sparse_matrix = csr_matrix(user_item_matrix)

# 🎯 ALS 모델 학습
als_model = AlternatingLeastSquares(factors=10, regularization=0.1, iterations=20)
als_model.fit(sparse_matrix)

# 🎯 사용자 1에게 추천 수행
user_id = 1
recommendations = als_model.recommend(user_id, sparse_matrix[user_id], N=3)
print("🔹 사용자 1에게 추천할 아이템:", recommendations)

c:\Users\Seonghyeon\anaconda3\envs\main\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Seonghyeon\anaconda3\envs\main\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
100%|██████████| 20/20 [00:00<00:00, 2652.86it/s]

🔹 사용자 1에게 추천할 아이템: (array([2, 1, 0]), array([ 3.2506272e-02,  2.7917266e-02, -3.4028235e+38], dtype=float32))


#### 2-3. NMF (Non-negative Matrix Factorization) 기반 추천

SVD는 음수 값을 허용하지만, NMF는 음수를 허용하지 않고 해석 가능성이 높은 행렬 분해 방식입니다.

##### NMF 추천 방식

In [10]:
from sklearn.decomposition import NMF

# NNMF 모델 적용 (2개의 잠재 요인 사용)
nmf = NMF(n_components=2, init='random', random_state=42)
U_nnmf = nmf.fit_transform(R)  # 사용자 잠재 요인 행렬
V_nnmf = nmf.components_  # 아이템 잠재 요인 행렬

# 예측 행렬 복원
R_nnmf_pred = np.dot(U_nnmf, V_nnmf)

print("=== NNMF 추천 예측 행렬 ===")
print(np.round(R_nnmf_pred, 2))

=== NNMF 추천 예측 행렬 ===
[[5.26 1.99 0.   1.46]
 [3.5  1.33 0.   0.97]
 [1.31 0.94 1.95 3.95]
 [0.98 0.72 1.53 3.08]
 [0.   0.65 2.84 5.22]]


##### 평가

In [ ]:
from surprise import NMF

# NMF 모델 생성 및 학습
nmf_model = NMF(n_factors=10, n_epochs=20)
nmf_model.fit(trainset)

# 평가
cross_validate(nmf_model, data, cv=5, verbose=True)

# ✅ NMF는 음수 값을 허용하지 않아 해석 가능성이 높음.
# ✅ SVD보다 결과 해석이 용이하지만, 데이터 특성에 따라 성능이 다를 수 있음.

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3832  0.9014  1.4583  2.0487  0.5627  1.2709  0.5081  
MAE (testset)     1.2868  0.7500  1.4570  1.9025  0.4546  1.1702  0.5140  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.38320264, 0.90138782, 1.45830882, 2.04874654, 0.56274335]),
 'test_mae': array([1.28683208, 0.75      , 1.45700357, 1.90252556, 0.45459099]),
 'fit_time': (0.0, 0.0, 0.0, 0.0, 0.001039743423461914),
 'test_time': (0.0, 0.0, 0.0, 0.0, 0.0)}

### 3. 성능 평가 및 모델 비교

#### 📌 추천 시스템 평가 지표
- RMSE (Root Mean Squared Error) : 예측 평점과 실제 평점 간의 차이
- Precision@K : 추천한 아이템 중 사용자가 실제로 소비한 아이템 비율
- Recall@K : 사용자가 소비한 아이템 중 추천한 아이템 비율

In [ ]:
from surprise import accuracy
rmse = accuracy.rmse(predictions)
print(f"🔹 RMSE: {rmse:.4f}")

# ✅ RMSE 값이 작을수록 추천 성능이 좋음.
# ✅ Precision@K, Recall@K를 활용하여 다양한 관점에서 평가 가능.

RMSE: 0.8517
🔹 RMSE: 0.8517


#### cf) Pytorch 기반 사용자-아이템 협업 필터링

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 사용자-아이템 평점 데이터
ratings = np.array([
    [1,1,5],
    [1,2,3],
    [2,1,4],
    [2,3,5],
    [3,2,2],
    [3,3,3],
    [4,1,5],
    [4,2,4]
])

# Pytorch Dataset 생성
class RatingDataset(Dataset):
    def __init__(self, ratings):
        self.users = torch.tensor(ratings[:,0]-1, dtype=torch.long) # 사용자 ID (0부터 시작)
        self.items = torch.tensor(ratings[:,1]-1, dtype=torch.long) # 아이템템 ID (0부터 시작)
        self.ratings = torch.tensor(ratings[:,2], dtype=torch.float32)
    
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

dataset = RatingDataset(ratings)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [15]:
# Pytorch 협업 필터링 모델 정의
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim = 10):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
    def forward(self, user, item):
        user_embedded = self.user_embedding(user)
        item_embedded = self.item_embedding(item)
        return (user_embedded * item_embedded).sum(1)

# 사용자 수와 아이템 수 계산
num_users = int(ratings[:, 0].max())
num_items = int(ratings[:, 1].max())

# 모델 초기화
model = MatrixFactorization(num_users, num_items)

# 🎯 4️⃣ 모델 학습
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 10
for epoch in range(num_epochs):
    for users, items, ratings in dataloader:
        optimizer.zero_grad()
        outputs = model(users, items)
        loss = criterion(outputs, ratings)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 19.4707
Epoch 2, Loss: 17.6865
Epoch 3, Loss: 25.6540
Epoch 4, Loss: 16.0960
Epoch 5, Loss: 29.6430
Epoch 6, Loss: 4.2679
Epoch 7, Loss: 19.5744
Epoch 8, Loss: 18.5817
Epoch 9, Loss: 16.1084
Epoch 10, Loss: 15.1465


In [16]:
# 🎯 5️⃣ 추천 예측
user_id = torch.tensor([0])
item_id = torch.tensor([1])
predicted_rating = model(user_id, item_id).item()
print(f"사용자 1이 아이템 2에 대한 예상 평점: {predicted_rating:.2f}")

사용자 1이 아이템 2에 대한 예상 평점: 2.79
